In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 

from pyspark.sql.functions import collect_list, concat_ws, udf, lit
from pyspark.sql.types import IntegerType

import pandas as pd
import numpy as np
from pyspark.sql.functions import pandas_udf




from datetime import datetime, timedelta, date, time
%pip install holidays
%pip install maya
import holidays
import maya

def __getholidaysbyyear(year):        
    ca_holidays = []
    for date, name in sorted(holidays.CA(prov='BC', years=year).items()):
        ca_holidays.append(date.strftime('%Y-%m-%d'))
    return ca_holidays    

def getholidays():
    ca_holidays = []
    currentyear = datetime.today().year
    years = [currentyear - 1, currentyear, currentyear + 1] 
    for year in years:
        ca_holidays.extend(__getholidaysbyyear(year))
    return ca_holidays

def isbusinessday(inpdate, holidays=None):
    _holidays = getholidays() if holidays is None else holidays
    if formatdate(inpdate) not in _holidays and __isweekday(inpdate) == True:
        return True
    return False

def __isweekday( inpdate):
    _inpdate = getdate(inpdate)
    if _inpdate.weekday() < 5:
        return True
    else:   
        return False 
    
def getbusinessdaysbetween( date1, date2=None):
    _holidays = getholidays()
    businessdays = 0
    date2 = date2 if date2 not in (None, '') else gettoday()
    _date1_date_fmt = getdate(date1)
    _date2_date_fmt = getdate(date2)
    __fromdate = _date1_date_fmt if _date1_date_fmt <= _date2_date_fmt else _date2_date_fmt
    __todate = _date2_date_fmt if _date2_date_fmt >= _date1_date_fmt else _date1_date_fmt
    __fromcalcdate =__fromdate
    while getdate(__fromcalcdate).date() < getdate(__todate).date():
        if isbusinessday(__fromcalcdate, _holidays) == True:
            businessdays += 1
        __fromcalcdate =  __fromcalcdate + timedelta(days=1)           
    return businessdays  

def getdate(inputdate):
    print("here3")
    return  datetime.strptime(inputdate, "%Y-%m-%d") if isinstance(inputdate, str) else datetime.combine(inputdate, time.min) if isinstance(inputdate, date) else inputdate   
    # return datetime.strptime(inputdate, "%Y-%m-%d") if isinstance(inputdate, str) else inputdate   


def gettoday(format=None):
    now_pst = maya.parse(maya.now()).datetime(to_timezone='America/Vancouver', naive=False)
    return __formatdate(now_pst, format)

def formatdate( input, format=None):
    _inpdate =  getdate(input)
    return __formatdate(_inpdate, format) 

def __formatdate(_inpdate, format):
    _format = format if format not in (None, '') else "%Y-%m-%d"
    return _inpdate.strftime(_format)


runcycleid = etl_helpers.start_run_cycle("factRequestCustomCalcFields")


# Get the current year
current_year = datetime.now().year

# Define the range of years: previous, current, next
year_range = [current_year - 1, current_year, current_year + 1]

# Generate the calendar
calendar = pd.date_range(
    start=f"{min(year_range)}-01-01",
    end=f"{max(year_range)}-12-31",
    freq="D"
)
bc_holidays = holidays.CA(prov="BC", years=year_range)
holiday_dates = set(bc_holidays.keys())

# Create DataFrame
df_calendar = pd.DataFrame({
    "date": calendar,
    "is_business_day": [(d.weekday() < 5 and d.date() not in holiday_dates) for d in calendar]
})

# Convert to Spark DataFrame and register
spark_df = spark.createDataFrame(df_calendar)
spark_df.createOrReplaceTempView("calendar")


try:

    query = f"""delete from  factrequestcustomcalcfields where sourceoftruth = 'FOIMOD' """;

    df = spark.sql(query)
    df.show()

    query = f"""

        SELECT 
          *,
          (
            SELECT COUNT(*) 
            FROM calendar c
            WHERE c.date BETWEEN CAST(final.firstduedate AS DATE) and current_date() 
              AND c.is_business_day = true
          ) AS daysoverdue
        FROM (

        select 
        *
        , case when requeststatuslabel = 'closed' then pageflagcount else 0 end as noofpagesreleased
        from 

        (SELECT *
                FROM (
                    SELECT 
                    duedate as firstduedate,
                    foirequest_id,
                    foiministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at asc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                ) sub
                WHERE rn = 1) sq
        join

        (SELECT *
                FROM (
                    SELECT 
                    duedate,
                    foiministryrequestid,
                    requeststatuslabel,
                    created_at,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at desc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                    -- where foiministryrequestid = 9
                ) sub
                WHERE rn = 1) sq2 on sq2.foiministryrequestid = sq.foiministryrequestid
        left join

        (SELECT *
                FROM (
                    SELECT 
                    created_at as lastonholddate,
                    foiministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at desc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                    where requeststatuslabel = 'onhold'
                ) sub
                WHERE rn = 1) sq3 on sq3.foiministryrequestid = sq.foiministryrequestid

        left join

        (SELECT *
                FROM (
                    SELECT 
                    created_at as firstonholddate,
                    foiministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at asc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                    where requeststatuslabel = 'onhold'
                ) sub
                WHERE rn = 1) sq4 on sq4.foiministryrequestid = sq.foiministryrequestid

        left join

        (SELECT count(recordid) as noofdocdelivered, ministryrequestid
                FROM (
                    SELECT 
                    ministryrequestid,
                    recordid, 
                    isactive,
                    version,
                    updated_at,
                    created_at,
                        ROW_NUMBER() OVER (
                            PARTITION BY recordid 
                            ORDER BY version desc
                        ) AS rn
                    FROM foi_mod.foirequestrecords
                    order by ministryrequestid
                    -- where ministryrequestid = 9
                ) sub
                WHERE rn = 1 and isactive = 't'
                group by ministryrequestid) sq5 on sq5.ministryrequestid = sq.foiministryrequestid

        left join

        (-- Step 1: Parse pageflag as JSON array of structs and explode it
                WITH exploded_flags AS (
                SELECT
                    documentid,
                    EXPLODE(from_json(pageflag, 'ARRAY<STRUCT<flagid: INT, page: INT, programareaid: ARRAY<INT>, other: ARRAY<STRING>>>')) AS flag
                FROM docreviewer.DocumentPageflags
                ),

                -- Step 2: Select the needed fields
                parsed_flags AS (
                SELECT
                    documentid,
                    flag.flagid
                FROM exploded_flags
                WHERE flag.flagid IS NOT NULL  -- only count flags that have a flagid
                )

                -- Step 3: Aggregate
                SELECT
                -- pf.documentid,
                -- pf.flagid,
                COUNT(*) AS pageflagcount,
                dpf.foiministryrequestid,
                'Y' AS isactive
                FROM parsed_flags pf
                JOIN docreviewer.DocumentPageflags dpf
                ON pf.documentid = dpf.documentid
                GROUP BY dpf.foiministryrequestid) sq6 on sq6.foiministryrequestid = sq.foiministryrequestid

        left join

        (SELECT *
                FROM (
                    SELECT 
                    ministryrequestid,
                    get_json_object(feedata, '$.estimatedelectronicpages') as electronicpageestimate,
                    get_json_object(feedata, '$.amountpaid') as feepaidamount,
                    get_json_object(feedata, '$.refundamount') as refundamount,
                    get_json_object(feedata, '$.refundamount') - get_json_object(feedata, '$.balanceremaining') as prepaymentamount,
                    get_json_object(feedata, '$.esetimatedhardcopypages') as physicalpageestimate,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY ministryrequestid 
                            ORDER BY created_at desc
                        ) AS rn
                    FROM foi_mod.foirequestcfrfees
                ) sub
                WHERE rn = 1) sq7 on sq7.ministryrequestid = sq.foiministryrequestid

        left join


        (select statusid, investigator, r.name as reason, rt.name as reviewtype, isjudicialreview, oipcno, foiministryrequest_id
        from foi_mod.foirequestoipc o
        join foi_mod.oipcreasons r on r.reasonid = r.reasonid
        join foi_mod.oipcreviewtypes rt on rt.reviewtypeid = o.reviewtypeid
        ) sq8 on sq8.foiministryrequest_id = sq.foiministryrequestid


        -- oi table not in prod yet
        -- (select iaorationale, ps.name
        -- from foi_mod.foiopeninformationrequests oi
        -- join openinfopublicationstatuses ps on ps.oipublicationstatus_id = oi.oipublicationstatus_id)

        left join

        (select foiministryrequestid, name as subject from foi_mod.foiministryrequestsubjectcodes msc
        join foi_mod.subjectcodes sc on msc.subjectcodeid = sc.subjectcodeid) sq9 on sq9.foiministryrequestid = sq.foiministryrequestid

        left join

        (select amountpaid, refundamount as applicationfeerefunded, foiministryrequestid
        from foi_mod.foirequestapplicationfees af
        join foi_mod.foirequests r on r.foirawrequestid = af.rawrequestid
        join foi_mod.foiministryrequests mr on r.foirequestid = mr.foirequest_id) sq10 on sq10.foiministryrequestid = sq.foiministryrequestid

        left join 
        (select watchedby, watchedbygroup, ministryrequestid
        from foi_mod.foirequestwatchers) sq11 on sq11.ministryrequestid = sq.foiministryrequestid

        left join
        (select sum(pagecount) as dedupepagecount, foiministryrequestid from docreviewer.documents d1
        join 
        (select rank1hash, min(d.documentid) as docid  from docreviewer.documenthashcodes dhc
        join docreviewer.documents d on d.documentid = dhc.documentid
        group by rank1hash) sq on sq.docid = d1.documentid
        group by foiministryrequestid) sq12 on sq12.foiministryrequestid = sq.foiministryrequestid
        ) final
        """

    print(query)

    df = spark.sql(query)
    print(df.count())
    df.show()


    # getdays = udf(getbusinessdaysbetween, IntegerType())

    
    # # Precompute BC holidays for a range of years
    # years = list(range(datetime.today().year - 1, datetime.today().year + 2))
    # bc_holidays = holidays.CA(prov='BC', years=years)
    # holiday_dates = set(pd.to_datetime(list(bc_holidays.keys())))

    # @pandas_udf(IntegerType())
    # def business_days_between(start_dates: pd.Series, end_dates: pd.Series) -> pd.Series:
    #     def count_business_days(start, end):
    #         if pd.isnull(start) or pd.isnull(end):
    #             return None
    #         start, end = min(start, end), max(start, end)
    #         all_days = pd.date_range(start=start, end=end, freq='D')
    #         weekdays = all_days[all_days.weekday < 5]
    #         business_days = weekdays[~weekdays.isin(holiday_dates)]
    #         return len(business_days)

    #     return start_dates.combine(end_dates, count_business_days)


    
    # @pandas_udf("integer") # specify the return type
    # def getdays(x: pd.Series) -> pd.Series:
    #     return getbusinessdaysbetween(date.today(), x)

    # print(df.first()['duedate'])
    # print(type(df.first()['duedate']))

    # df = df.withColumn("daysoverdue", getdays(lit(date.today()), df['firstduedate']))
    # df = df.withColumn("passduedays", getdays(lit(date.today()), df['firstduedate']))

    # df = df.withColumn("passduedays", business_days_between(lit(date.today()), df['firstduedate']))

    # df.show(df.count(), truncate=False)



    # order of columns here is important!
    df_mapped = df.selectExpr( 
        "foirequest_id as foirequestid",
        f"{runcycleid} as runcycleid",
        "'' as identityverification",
        "'' as previousfoirequest",
        "amountpaid as applicationfeepaid",
        "applicationfeerefunded as applicationfeerefunded",
        "'' as applicationfeetransferred",
        "'' as linkedrequests",
        "double(feepaidamount) as feepaidamount",
        "double(refundamount) as refundamount",
        "prepaymentamount as prepaymentamount",
        "physicalpageestimate as physicalpageestimate",
        "electronicpageestimate as electronicpageestimate",
        "statusid as customfieldstatus",
        "watchedby as secondaryusers",
        "'' as applicantfilereference",
        "'' as coordinatednrresponsereqd",
        "investigator AS portfolioofficer",
        "reason AS reason",
        "reviewtype AS reviewtype",
        "isjudicialreview AS judicialreview",
        "oipcno AS oipcno",
        "'' AS publicationreason",
        "'' AS publication",
        "'' AS crossgovtno",
        "subject as subject",
        "created_at AS currentactivitydate",
        "noofdocdelivered AS noofdocdelivered",
        "lastonholddate - firstonholddate AS onholddays",
        "created_at <= duedate as countontime",
        "created_at > duedate as countoverdue",
        "daysoverdue AS daysoverdue",
        "daysoverdue as passduedays",
        "pageflagcount as noofpagesreviewed",
        "pageflagcount as noofpagesreleased",
        "dedupepagecount AS noofpagesdeduplicated",
        "'FOIMOD' as sourceoftruth",
    )
    df_mapped.show()    
    df_mapped.printSchema()
    df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestcustomcalcfields")  
    etl_helpers.end_run_cycle(runcycleid, 't', "factRequestCustomCalcFields")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestCustomCalcFields", "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:    
    if (str(e) == "no changes for today"):
        # print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factRequestCustomCalcFields")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestCustomCalcFields", f"An error occurred: {e}")
        raise Exception("notebook failed") from e